In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
school_data_tofind = "Resources/Chicago_Public_Schools_-_School_Progress_Reports_SY1819.csv"
school_data_tofind2 = "Resources/Chicago_Public_Schools_-_School_Admissions_Information_SY1819.csv"
school_data_tofind3 = "Resources/employeepositionroster_09302018.csv"
school_data_tofind4 = "Resources/employeepositionroster_09302019.csv"

In [3]:
school_data1 = pd.read_csv(school_data_tofind)
school_data2 = pd.read_csv(school_data_tofind2)
school_data3 = pd.read_csv(school_data_tofind3)
school_data4 = pd.read_csv(school_data_tofind4)

In [6]:
#partition data to creat df with only high school data
high_school_data1 = school_data1.loc[(school_data1['Primary_Category']== "HS")]
Charter_high = high_school_data1.loc[(high_school_data1['School_Type']== "Charter")]
#partition data to creat df with only high school data
Neighborhood_high = high_school_data1.loc[(high_school_data1['School_Type']== "Neighborhood")]


In [21]:
Charter_high["Attainment_SAT_Grade_11_School_Pct"].var()

476.02342090395473

In [7]:
Charter_high["Attainment_SAT_Grade_11_School_Pct"].mean()

20.971666666666668

In [8]:
Neighborhood_high["Attainment_SAT_Grade_11_School_Pct"].mean()

19.504545454545454

In [16]:
critical_value = stats.chi2.ppf(q = 0.95, df = 1)
critical_value

3.841458820694124

In [19]:
# Run the chi square test with stats.chisquare()
stats.chisquare(20.971,19.505)

IndexError: tuple index out of range

In [9]:
Charter_high["Growth_SAT_Math_Grade_11_School_Pct"].mean()

61.853658536585364

In [10]:
Neighborhood_high["Growth_SAT_Math_Grade_11_School_Pct"].mean()

51.22727272727273

In [11]:
Neighborhood_high["Growth_SAT_Reading_Grade_11_School_Pct"].mean()

47.06818181818182

In [12]:
Charter_high["Growth_SAT_Reading_Grade_11_School_Pct"].mean()

45.31707317073171

In [ ]:
# # Start by getting the last (greatest) timepoint for each mouse
# grouped_Max = drug_data.groupby('Mouse ID')
# Maxes = grouped_Max['Timepoint'].max()
# # Merge this group df with the original dataframe to get the tumor volume at the last timepoint
# Max_Vol = pd.merge(Maxes, drug_data, on=['Mouse ID', 'Timepoint'], how='left')
# Max_Vol